In [1]:
import config
import sqlite3
import graphviz
import data_load
import pandas as pd
from dtreeviz.trees import dtreeviz
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [2]:
conn = sqlite3.connect(f'{config.PATH_DATABASES}/PARTOS/partos.db')

In [3]:
pd.read_sql_query('select * from sinasc limit 5', con=conn)

,cnes_hospital,idade,nivel_escolaridade,n_gestados,data,apgar1,apgar5,raca_cor,sexo_nascido,raca_cor_nascido,local_nascimento,tipo_parto,consultas_pre_natal,hosp_cod_regiao_saude,hosp_cod_municipio,res_cod_regiao_saude,res_cod_municipio,ano
0,2515598,24,4,1,7052010,9.0,10.0,None,Masculino,Branca,Hospital,Cesáreo,7 e mais,1104,110002,1201,120040,2010
1,2798484,20,3,1,23022010,8.0,9.0,None,Masculino,Branca,Hospital,Vaginal,7 e mais,1103,110030,1201,120040,2010
2,5701929,21,4,1,18042010,9.0,10.0,None,Feminino,Parda,Hospital,Vaginal,de 4 a 6,1201,120001,1201,120001,2010
3,5701929,31,3,1,17032010,9.0,10.0,None,Feminino,Parda,Hospital,Vaginal,de 4 a 6,1201,120001,1201,120001,2010
4,5701929,23,3,1,18032010,7.0,9.0,None,Masculino,Parda,Hospital,Vaginal,7 e mais,1201,120001,1201,120001,2010


In [4]:
path_query = 'queries/info/perfil.sql'
with open(path_query) as f:
	query = f.read()
# df = pd.read_sql_query(query, con=conn)
# df

In [5]:
perfil_path = 'data/consult/perfil_gestante.csv.xz'
# df.to_csv(perfil_path, index=False)
df = pd.read_csv(perfil_path)

In [6]:
path_query = 'queries/info/gestantes.sql'
with open(path_query) as f:
	query = f.read()
# df = data_load.df_query(query)
df

,idade,nivel_escolaridade,raca_cor,tipo_parto,res_cod_municipio,hosp_cod_municipio,res_cod_regiao_saude,hosp_cod_regiao_saude,raca_cor_nascido,apgar1,apgar5,records
0,21,4,4.0,Vaginal,130260,130260,1301,1301,Parda,9.0,10.0,241
1,33,5,1.0,Cesáreo,330455,330455,3305,3305,Branca,9.0,10.0,238
2,20,4,4.0,Vaginal,130260,130260,1301,1301,Parda,9.0,10.0,229
3,19,4,4.0,Vaginal,130260,130260,1301,1301,Parda,9.0,10.0,226
4,22,4,4.0,Vaginal,130260,130260,1301,1301,Parda,9.0,10.0,222
...,...,...,...,...,...,...,...,...,...,...,...,...
889959,49,4,4.0,Vaginal,230590,230410,2315,2315,Parda,8.0,9.0,1
889960,49,4,4.0,Vaginal,353800,353800,3524,3524,Parda,9.0,9.0,1
889961,49,5,1.0,Cesáreo,350950,350950,3512,3512,Branca,9.0,10.0,1
889962,49,5,1.0,Cesáreo,354580,350160,3512,3512,Branca,3.0,5.0,1


### Conversões de Tipagem

In [7]:
# df['consultas'] = df['consultas'].astype(int)
df['nivel_escolaridade'] = df['nivel_escolaridade'].astype(int)
df['apgar1'] = df['apgar1'].astype(int)
df['apgar5'] = df['apgar5'].astype(int)

### Idade

[(11.964, 24.0] < (24.0, 36.0] < (36.0, 48.0]]

Bins

In [8]:
df['idade'].unique()

array([21, 33, 20, 19, 22, 31, 32, 30, 25, 18, 17, 24, 26, 23, 29, 34, 35,
       28, 36, 16, 27, 37, 38, 15, 39, 40, 14, 42, 41, 13, 43, 44, 12, 45,
       11, 46, 47, 10, 48, 49])

In [9]:
pd.cut(df['idade'], 3)

0         (9.961, 23.0]
1          (23.0, 36.0]
2         (9.961, 23.0]
3         (9.961, 23.0]
4         (9.961, 23.0]
              ...      
889959     (36.0, 49.0]
889960     (36.0, 49.0]
889961     (36.0, 49.0]
889962     (36.0, 49.0]
889963     (36.0, 49.0]
Name: idade, Length: 889964, dtype: category
Categories (3, interval[float64, right]): [(9.961, 23.0] < (23.0, 36.0] < (36.0, 49.0]]

In [10]:
df['idade'] = pd.cut(df['idade'], 3)

### Raça/Cor

Map

In [11]:
df['raca_cor'] = df['raca_cor'].map(
	{1: 'Branca', 2: 'Preta', 3: 'Amarela', 4: 'Parda', 5: 'Indígena'}
)

### Criticidade

- 0: res_cod_municipio == hosp_cod_municipio
- 1: (res_cod_municipio != hosp_cod_municipio) and (res_cod_regiao_saude == hosp_cod_regiao_saude)
- 2: res_cod_regiao_saude != hosp_cod_regiao_saude

In [12]:
df['criticidade_0'] = df['res_cod_municipio'] == df['hosp_cod_municipio']
df['criticidade_1'] = (df['res_cod_municipio'] != df['hosp_cod_municipio']) & (df['res_cod_regiao_saude'] == df['hosp_cod_regiao_saude'])
df['criticidade_2'] = df['res_cod_regiao_saude'] != df['hosp_cod_regiao_saude']

### Grupo SocioEconômico

de 1 a 5 com base na região de saúde do município

In [13]:
municipios = pd.read_csv('data/consult/health_regions.csv')
municipios = {row['Cód IBGE'] : row['Grupo'] for _, row in municipios.iterrows()}

In [14]:
df['grupo'] = df['res_cod_municipio'].map(municipios)

In [15]:
df.grupo.unique()

array([ 4.,  5.,  2.,  3.,  1.,  0., nan])

### Remover variáveis que não serão usadas

- res_cod_municipio
- hosp_cod_municipio
- res_cod_regiao_saude
- hosp_cod_regiao_saude

In [16]:
# - res_cod_municipio
# - hosp_cod_municipio
# - res_cod_regiao_saude
# - hosp_cod_regiao_saude

In [17]:
df.drop(['records', 'grupo',
	'res_cod_municipio', 'hosp_cod_municipio',
	'res_cod_regiao_saude', 'hosp_cod_regiao_saude',
], axis=1, inplace=True)

### Resultado

In [18]:
df = pd.get_dummies(df)

In [19]:
df

,nivel_escolaridade,apgar1,apgar5,criticidade_0,criticidade_1,criticidade_2,"idade_(9.961, 23.0]","idade_(23.0, 36.0]","idade_(36.0, 49.0]",raca_cor_Amarela,...,raca_cor_Indígena,raca_cor_Parda,raca_cor_Preta,tipo_parto_Cesáreo,tipo_parto_Vaginal,raca_cor_nascido_Amarela,raca_cor_nascido_Branca,raca_cor_nascido_Indígena,raca_cor_nascido_Parda,raca_cor_nascido_Preta
0,4,9,10,True,False,False,1,0,0,0,...,0,1,0,0,1,0,0,0,1,0
1,5,9,10,True,False,False,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,4,9,10,True,False,False,1,0,0,0,...,0,1,0,0,1,0,0,0,1,0
3,4,9,10,True,False,False,1,0,0,0,...,0,1,0,0,1,0,0,0,1,0
4,4,9,10,True,False,False,1,0,0,0,...,0,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889959,4,8,9,False,True,False,0,0,1,0,...,0,1,0,0,1,0,0,0,1,0
889960,4,9,9,True,False,False,0,0,1,0,...,0,1,0,0,1,0,0,0,1,0
889961,5,9,10,True,False,False,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
889962,5,3,5,False,True,False,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0


In [20]:
for col in df.columns:
	print(col)

nivel_escolaridade
apgar1
apgar5
criticidade_0
criticidade_1
criticidade_2
idade_(9.961, 23.0]
idade_(23.0, 36.0]
idade_(36.0, 49.0]
raca_cor_Amarela
raca_cor_Branca
raca_cor_Indígena
raca_cor_Parda
raca_cor_Preta
tipo_parto_Cesáreo
tipo_parto_Vaginal
raca_cor_nascido_Amarela
raca_cor_nascido_Branca
raca_cor_nascido_Indígena
raca_cor_nascido_Parda
raca_cor_nascido_Preta


In [21]:
df['criticidade_0'] = df['criticidade_0'].astype(bool)
df['criticidade_1'] = df['criticidade_1'].astype(bool)
df['criticidade_2'] = df['criticidade_2'].astype(bool)
df['idade_(9.961, 23.0]'] = df['idade_(9.961, 23.0]'].astype(bool)
df['idade_(23.0, 36.0]'] = df['idade_(23.0, 36.0]'].astype(bool)
df['idade_(36.0, 49.0]'] = df['idade_(36.0, 49.0]'].astype(bool)
df['tipo_parto_Cesáreo'] = df['tipo_parto_Cesáreo'].astype(bool)
df['tipo_parto_Vaginal'] = df['tipo_parto_Vaginal'].astype(bool)
df['raca_cor_Amarela'] = df['raca_cor_Amarela'].astype(bool)
df['raca_cor_Branca'] = df['raca_cor_Branca'].astype(bool)
df['raca_cor_Indígena'] = df['raca_cor_Indígena'].astype(bool)
df['raca_cor_Parda'] = df['raca_cor_Parda'].astype(bool)
df['raca_cor_Preta'] = df['raca_cor_Preta'].astype(bool)
df['raca_cor_nascido_Preta'] = df['raca_cor_nascido_Preta'].astype(bool)
df['raca_cor_nascido_Amarela'] = df['raca_cor_nascido_Amarela'].astype(bool)
df['raca_cor_nascido_Branca'] = df['raca_cor_nascido_Branca'].astype(bool)
df['raca_cor_nascido_Indígena'] = df['raca_cor_nascido_Indígena'].astype(bool)
df['raca_cor_nascido_Parda'] = df['raca_cor_nascido_Parda'].astype(bool)
df['raca_cor_nascido_Preta'] = df['raca_cor_nascido_Preta'].astype(bool)

### Separando X, y

In [22]:
criticidade = [
	'criticidade_0', 'criticidade_1', 'criticidade_2'
]

In [23]:
X = df.drop(criticidade, axis=1)
y = df[criticidade]

In [24]:
X

,nivel_escolaridade,apgar1,apgar5,"idade_(9.961, 23.0]","idade_(23.0, 36.0]","idade_(36.0, 49.0]",raca_cor_Amarela,raca_cor_Branca,raca_cor_Indígena,raca_cor_Parda,raca_cor_Preta,tipo_parto_Cesáreo,tipo_parto_Vaginal,raca_cor_nascido_Amarela,raca_cor_nascido_Branca,raca_cor_nascido_Indígena,raca_cor_nascido_Parda,raca_cor_nascido_Preta
0,4,9,10,True,False,False,False,False,False,True,False,False,True,False,False,False,True,False
1,5,9,10,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False
2,4,9,10,True,False,False,False,False,False,True,False,False,True,False,False,False,True,False
3,4,9,10,True,False,False,False,False,False,True,False,False,True,False,False,False,True,False
4,4,9,10,True,False,False,False,False,False,True,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889959,4,8,9,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False
889960,4,9,9,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False
889961,5,9,10,False,False,True,False,True,False,False,False,True,False,False,True,False,False,False
889962,5,3,5,False,False,True,False,True,False,False,False,True,False,False,True,False,False,False


In [25]:
y

,criticidade_0,criticidade_1,criticidade_2
0,True,False,False
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False
...,...,...,...
889959,False,True,False
889960,True,False,False
889961,True,False,False
889962,False,True,False


### Árvore de Decisão

In [26]:
clf = DecisionTreeClassifier(
	random_state=0,
	criterion='entropy',
	max_depth=5
)
clf.fit(X, y)

DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)

In [27]:
estrutura = export_graphviz(
    decision_tree = clf,
    out_file = None,
    feature_names = X.columns,
    filled = True
)
graph = graphviz.Source(estrutura)
png_bytes = graph.pipe(format='png')
with open('dtree_pipe.png','wb') as f:
    f.write(png_bytes)

In [28]:
viz = dtreeviz(clf)
viz

TypeError: unhashable type: 'numpy.ndarray'